### This notebook analyzes the output of the few-shot evaluation
i.e. whether it follows the format of the few-shot evaluation

In [1]:
import numpy as np
import json
import pandas as pd
import json
import os
import os
import json


def remove_underline(x):
    return x.replace("_", "")

# read the output logs file
with open('/home/aiops/zhuty/tinyllama/scripts/fewshot_eval/outputs/logs', 'r') as file:
    data = file.readlines()
data = [ json.loads(x) for x in data]
print(data)

In [ ]:
# BASE_PATH = '/home/aiops/zhuty/tinyllama/scripts//fewshot_eval/fewshot_out_mar27'
BASE_PATH = '/home/aiops/zhuty/tinyllama/scripts/fewshot_eval/outputs'
commands = []
data = []

NSHOTS=24
shot_num = NSHOTS
# iter_name = "iter-380000-ckpt-step-47500_hf"
# size="1b"

high_priority=False
# iter_name = "iter-160000-ckpt-step-40000_hf"
# size="360M"
# iter_name = "iter-240000-ckpt-step-60000_hf"
# size="120M"
iter_name = "iter-380000-ckpt-step-47500_hf"
size="1b"



TASK_CLASS="icl"

class_to_tasks_mapping = {"icl": ['agnews' ,'amazon' ,'dbpedia' ,'sst2' ,'tweet_hate' ,'tweet_offensive' ,'yelp' ],
                          "obqa": [ "squad", "nq_obqa", "tq_obqa" ,"hotpotqa"],
                          "cbqa": ["tq", "nq"]
                          }

class_to_seed_mapping = {"icl": range(42, 58),
                         "obqa": range(42, 46+1),
                         "cbqa": range(42, 46 + 1)
                         }
all_data = {}
for task_name in  class_to_tasks_mapping[TASK_CLASS]:
    for model_name in ['cc', 'cc_merged_v1', 'cc_merged_v2' ,'cc_merged_v3']:
        for seed_num in class_to_seed_mapping[TASK_CLASS]:
            full_name = f"tiny_LLaMA_{size}_8k_{model_name}_8k-{iter_name}"
            # full_name = "TinyLlama/TinyLlama-1.1B-step-50K-105b"
            result_path = os.path.join(BASE_PATH,full_name, f'{task_name}_{shot_num}_{seed_num}_prompts_and_preds.json')
            if not os.path.exists(result_path):
                print(f"File {result_path} does not exist")
                commands.append(f"python /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/eval.py --model_name {full_name} --task_name {task_name} --n_shot {shot_num} --seed {seed_num} --high_priority {high_priority}")
                continue
            else:
                print("Existing")
                curr_data = json.load(open(result_path, 'r'))
                all_data[(task_name, model_name, seed_num)] = curr_data
                # curr_prompts = [x['prompt'] for x in curr_data['prompts']]
                # curr_preds = [x['pred'] for x in curr_data['preds']]



Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing
Existing


In [ ]:
all_data.keys()

In [ ]:
all_data[(('amazon', 'cc', 52))]['preds']

In [ ]:
def is_generated_label(output_string, labels):
    gen_label = output_string.split("\n")[0]
    if gen_label in labels:
        return True
    return False

In [ ]:
# task_to_labels = {"agnews": {0: "world", 1: "sports", 2: "business", 3: "science"},
# "amazon": {0: "negative", 1: "positive"},
# "yelp": {0: "negative", 1: "positive"},
# "sst2": {0: "positive", 1: "negative"},
# "tweet_hate": {0: "Non-hate", 1: "Hate"},
# "tweet_offensive":  {0: "Non-hate", 1: "Hate"},
#                   }
#
# task_to_labels['dbpedia'] = {idx:name for idx,name in enumerate(["Company", "EducationalInstitution", "Artist", "Athlete", "OfficeHolder", "MeanOfTransportation",
#              "Building", "NaturalPlace", "Village", "Animal", "Plant", "Album", "Film", "WrittenWork", ])}

task_to_labels = {"agnews": {0: "M", 1: "N", 2: "Q", 3: "P"},
"amazon": {0: "foo", 1: "bar"},
"yelp": {0: "foo", 1: "bar"},
"sst2": {0: "foo", 1: "bar"},
"tweet_hate": {0: "foo", 1: "bar"},
"tweet_offensive":  {0: "foo", 1: "bar"},
                  }

task_to_labels['dbpedia'] = {idx:name for idx,name in enumerate( ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N"])}

In [ ]:
all_data[(('tweet_offensive', 'cc_merged_v3', 52))]['prompts'][0]

In [ ]:
task_model_percentage = {}
task_model_wrong_cases = {}
# take the average over all the seeds
for key, data in all_data.items():
    # data = all_data[(('tweet_offensive', 'cc', 52))]['preds']
    task_name, model_name, seed_num = key
    labels = task_to_labels[task_name].values()
    data = data['preds']

    for x in data:
        if (task_name, model_name) not in task_model_percentage:
                task_model_percentage[(task_name, model_name)] = []
        if (task_name, model_name) not in task_model_wrong_cases:
            task_model_wrong_cases[(task_name, model_name)] = []
        if is_generated_label(x, labels):
            task_model_percentage[(task_name, model_name)].append(1)
        else:
            task_model_wrong_cases[(task_name, model_name)].append(x)
            task_model_percentage[(task_name, model_name)].append(0)

In [ ]:
# print the average of correct labels
for key, data in task_model_percentage.items():
    print(key, np.mean(data))

In [ ]:
(task_model_wrong_cases[('tweet_offensive', 'cc_merged_v3')])

In [21]:
assert len(commands) == 0 , "missing commands {}".format( len(commands))


In [23]:
def extract_step(x):
    # strip the _hf
    x = x[:-3]
    return int(x.split('-')[-1])

def extract_model_name(x):
    """
    Sample input: /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_1b_8k_cc_8k/iter-200000-ckpt-step-25000_hf
    Sample output: tiny_LLaMA_1b_8k
    Args:
        x:
    Returns:

    """
    if '/home/aiops/zhuty' in x:
        folder_name = x.split('/')[-2] # tiny_LLaMA_1b_8k_cc_8k
        return ("_").join(folder_name.split('_')[:4]) # tiny_LLaMA_1b_8k
    else:
        # sample model name: tyzhu/tiny_LLaMA_120M_8k_cc_8k_iter-240000-ckpt-step-60000_hf
        return x.strip("tyzhu/").split("_cc")[0]

def extract_pretrain_dataset(x):
    """
    Sample input: /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_1b_8k_cc_8k/iter-200000-ckpt-step-25000_hf
    Sample output: cc_8k
    Args:
        x:
    Returns:

    """
    if '/home/aiops/zhuty' in x:
        folder_name = x.split('/')[-2] # tiny_LLaMA_1b_8k_cc_8k
        return ("_").join(folder_name.split('_')[4:]) # cc_8k
    else:
        # sample model name: tyzhu/tiny_LLaMA_120M_8k_cc_8k_iter-240000-ckpt-step-60000_hf
        return x.strip("tyzhu/").split("_8k_")[1]
